In [1]:
import time
import jwt
import requests
from numpy import dot
from numpy.linalg import norm


KEY = '83e5bc58555d8bac289e27bac50f8afc.Khk1JjCxb8MJN8Mi'

# 实际KEY，过期时间


def generate_token(apikey: str, exp_seconds: int):
    try:
        id, secret = apikey.split('.')
    except Exception as e:
        raise Exception('invalid apikey', e)

    payload = {
        'api_key': id,
        'exp': int(round(time.time() * 1000)) + exp_seconds * 1000,
        'timestamp': int(round(time.time() * 1000)),
    }
    return jwt.encode(
        payload,
        secret,
        algorithm='HS256',
        headers={'alg': 'HS256', 'sign_type': 'SIGN'},
    )

In [2]:
def ask_glm(content):
    url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
    headers = {
      'Content-Type': 'application/json',
      'Authorization': generate_token(KEY, 1000)
    }

    data = {
        "model": "glm-4",
        "messages": [{"role": "user", "content": content}]
    }

    response = requests.post(url, headers=headers, json=data)
    return response.json()

In [3]:
ask_glm('你好，请解释雪崩的原理。')

{'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': '雪崩原理是半导体物理中的一个重要概念，主要描述在某些特定条件下，半导体器件中的载流子（电子和空穴）数量会因为碰撞电离效应而急剧增加，形成一个类似雪崩的过程。\n\n在半导体PN结中，当施加的反向电压增加时，空间电荷区中的电场也会随之增强。当这个电场强度达到一定程度，通过空间电荷区的载流子（电子或空穴）会因为电场的作用而获得足够的能量。当这些载流子与半导体材料中的原子发生碰撞时，它们可以拥有足够的动能去打破共价键，从而产生新的电子-空穴对。\n\n这个过程称为碰撞电离。新产生的电子和空穴同样会在电场作用下加速，获得更多的能量，然后与更多的原子发生碰撞，产生更多的电子-空穴对。这种连锁反应就像雪崩一样，一旦开始就迅速扩大，导致在很短的时间内产生大量的载流子。\n\n当反向电压继续增加，最终会达到一个击穿电压（UB），这时产生的电子-空穴对数量多到足以导致反向电流急剧增大，形成所谓的雪崩击穿。掺杂浓度较低的PN结由于空间电荷区较宽，载流子在其中运动的时间更长，与原子发生碰撞的机会更多，因此更容易发生雪崩击穿。\n\n雪崩击穿的现象被广泛应用于雪崩二极管的设计中。雪崩二极管利用这一原理，在特定条件下可以提供一个高反向耐压的特性，并且由于其负阻特性，可以用于放大微波信号等应用。然而，由于雪崩过程中载流子倍增的无规则性，雪崩二极管在提供这些特性的同时，也会伴随较大的噪声，这种噪声被称为雪崩噪声。',
    'role': 'assistant'}}],
 'created': 1708855891,
 'id': '8415032666110293167',
 'model': 'glm-4',
 'request_id': '8415032666110293167',
 'usage': {'completion_tokens': 355,
  'prompt_tokens': 779,
  'total_tokens': 1134}}

In [4]:
import json

In [6]:
questions = json.load(open('./submit_6.json'))

In [7]:
refine_list = []
failed_answer = '无法回答问题'

for i in range(len(questions)):
    answer = questions[i]['answer']
    if answer.find(failed_answer) != -1:
        refine_list.append(i)

In [9]:
len(refine_list)

51

In [16]:
prompt = '你是一个专业且诚实的汽车导购，也就是说你可以进行合理的推理。现在给你一份资料：\n{}\n，这份资料来自于第{}页，用户的问题是：{} 。如果资料中找不到答案，请回答：结合给定的资料，无法回答问题；否则你应该准确地回答问题。'

In [17]:
prompt.format(1, 1, 1)

'你是一个专业且诚实的汽车导购，也就是说你可以进行合理的推理。现在给你一份资料：\n1\n，这份资料来自于第1页，用户的问题是：1 。如果资料中找不到答案，请回答：结合给定的资料，无法回答问题；否则你应该准确地回答问题。'

In [14]:
import pdfplumber

pdf = pdfplumber.open('./初赛训练数据集.pdf')

pdf_content = []

for idx, content in enumerate(pdf.pages):
    pdf_content.append(
        {
            'page_idx': str(idx + 1),
            'page_content': content.extract_text()
        }
    )

In [24]:
refine_list

[0,
 2,
 6,
 8,
 11,
 13,
 16,
 25,
 34,
 38,
 40,
 48,
 50,
 66,
 74,
 77,
 80,
 84,
 86,
 92,
 93,
 94,
 107,
 143,
 147,
 174,
 183,
 184,
 185,
 193,
 201,
 206,
 207,
 219,
 222,
 223,
 226,
 229,
 231,
 236,
 243,
 252,
 257,
 258,
 263,
 267,
 275,
 279,
 285,
 297,
 298]

In [25]:
import torch
import transformers
from transformers import  AutoTokenizer, AutoModelForSequenceClassification, AutoModel

rerank_tokenizer = AutoTokenizer.from_pretrained('../BAAI/bge-reranker-base/')
rerank_model = AutoModelForSequenceClassification.from_pretrained('../BAAI/bge-reranker-base/')

embedding_tokenizer = AutoTokenizer.from_pretrained('../BAAI/bge-large-zh-v1.5/')
embedding_model = AutoModel.from_pretrained('../BAAI/bge-large-zh-v1.5/')

In [26]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [27]:
device

device(type='cuda', index=0)

In [28]:
rerank_model.to(device)
embedding_model.to(device)
rerank_model.eval()
embedding_model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inpl

In [29]:
import jieba

pdf_content_words = [jieba.lcut(x['page_content']) for x in pdf_content]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ASUS\AppData\Local\Temp\jieba.cache
Loading model cost 0.516 seconds.
Prefix dict has been built successfully.


In [30]:
from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(pdf_content_words)

In [31]:
questions_embedding = []

with torch.no_grad():
    for query_idx in range(len(questions)):
        encoded_query = embedding_tokenizer(questions[query_idx]['question'], truncation=True, padding=True, return_tensors='pt', max_length=512)
        encoded_query = {k: v.cuda() for k, v in encoded_query.items()}
        output = embedding_model(**encoded_query)
        query_embedding = output[0][:, 0]
        questions_embedding.append(query_embedding)

questions_embedding = torch.concatenate(questions_embedding)
questions_embedding.shape

torch.Size([301, 1024])

In [32]:
encoded_docs = embedding_tokenizer([x['page_content'] for x in pdf_content], truncation=True, padding=True, return_tensors='pt', max_length=512)   

with torch.no_grad():
    encoded_docs = {k: v.cuda() for k, v in encoded_docs.items()}
    outputs = embedding_model(**encoded_docs)
    docs_embedding = outputs[0][:, 0]

docs_embedding.shape

torch.Size([354, 1024])

In [33]:
torch.cuda.empty_cache() 

In [34]:
import torch.nn.functional as F

query_2_doc_similarity = F.normalize(questions_embedding) @ F.normalize(docs_embedding).T

In [35]:
query_2_doc_similarity.shape

torch.Size([301, 354])

In [62]:
prompt = '你是一个专业且诚实的汽车导购，这意味着你可以根据资料进行合理的推理。用户的问题是：{}。\n \
    对于该问题，有三笔资料，分别是来自第{}页的资料：{}；\n来自第{}页的资料：{}；\n来自第{}页的资料: {}, 请结合这三笔资料给出回答。如果这三笔资料都没有相关的答案，请如实回答： \
        结合给定的资料，无法回答问题。'

In [67]:
import numpy as np

for query_idx in refine_list:
    lexical_page_scores = bm25.get_scores(jieba.lcut(questions[query_idx]['question']))
    lexical_top3_score_page_indexes = lexical_page_scores.argsort()[-3:].tolist()

    semantic_page_scores = query_2_doc_similarity[query_idx].cpu().numpy()
    semantic_top3_score_page_indexes = semantic_page_scores.argsort()[-3:].tolist()

    candidates = list(set(lexical_top3_score_page_indexes + semantic_top3_score_page_indexes))
    pairs = [[questions[query_idx]['question'], pdf_content[page_idx]['page_content']] for page_idx in candidates]
    candidates = np.array(candidates)

    inputs = rerank_tokenizer(pairs, padding=True, truncation=True, max_length=512, return_tensors='pt')
    with torch.no_grad():
        inputs = {k: v.to(device) for k, v in inputs.items()}
        scores = rerank_model(**inputs, return_dict=True).logits.view(-1, ).float()

    # print(scores.shape)
    top3_match_indexes = candidates[scores.cpu().numpy().argsort()[-3:]]
    # print(top3_match_indexes)
    # break

    content = prompt.format(questions[query_idx]['question'], 
                                       top3_match_indexes[0] + 1, pdf_content[top3_match_indexes[0]],
                                       top3_match_indexes[1] + 1, pdf_content[top3_match_indexes[1]],
                                       top3_match_indexes[2] + 1, pdf_content[top3_match_indexes[2]])

    new_answer = ask_glm(content)['choices'][0]['message']['content']
    questions[query_idx]['answer'] = new_answer
    questions[query_idx]['reference'] = 'page_' + str(top3_match_indexes[-1] + 1)    
    # print(questions[query_idx]['question'], new_answer, content)
    # break

    # print(questions[query_idx]['question'], candidates, candidates[top3_match_indexes])


In [68]:
for i in refine_list:
    print(questions[i])

{'question': '“前排座椅通风”的相关内容在第几页？', 'answer': '根据提供的资料，关于“前排座椅通风”的相关内容出现在第116页和第117页的资料中。这两页资料详细描述了如何通过中央显示屏和控制界面来设置驾驶员侧和副驾驶员侧的座椅通风强度、开关控制以及通风时间。因此，可以回答用户的问题：“前排座椅通风”的相关内容在第116页和第117页。', 'reference': 'page_115'}
{'question': '“打开前机舱盖”的相关信息在第几页？', 'answer': '根据提供的资料，关于“打开前机舱盖”的相关信息可以在第307页找到。该页详细描述了如何打开前机舱盖的步骤，并提醒用户在操作前确保前机舱盖打开区域无障碍物。因此，可以明确回答用户的问题：“打开前机舱盖”的相关信息在第307页。', 'reference': 'page_308'}
{'question': '问题：事件数据记录系统（EDR）中的数据是否可以被黑客利用进行恶意攻击？', 'answer': '根据提供的资料，事件数据记录系统（EDR）主要用于记录车辆发生碰撞事故时的相关信息，例如车辆的行驶速度、制动状态等。这些信息有助于事故分析、产品质量提升和符合法律法规的要求。\n\n从资料中可以看出，EDR的数据读取需要使用车辆制造商提供的专用数据读取设备，并且只有经过授权的中心才能进行读取。此外，第12页的资料中提到，Lynk&Co领克汽车配备的远程监控系统定期将数据上传至制造商，但并未提及EDR数据可以通过远程监控系统被访问。\n\n就资料所提供的信息而言，没有明确说明EDR数据能否被黑客利用进行恶意攻击。但是，考虑到EDR的数据读取需要专用设备和授权，这在一定程度上增加了数据的安全性，减少了被黑客利用的风险。\n\n综上所述，结合给定的资料，无法直接回答问题“EDR中的数据是否可以被黑客利用进行恶意攻击”。如果需要更准确的答案，可能需要查阅更多关于EDR安全性的专业资料或咨询相关领域的专家。', 'reference': 'page_12'}
{'question': '请问，如果车辆报废后，原车主是否还能使用该车辆的智能互联服务？', 'answer': '根据提供的资料，如果车辆报废后，原车主将不能使用该车辆的智能互联服务。在第14页的资料中明确指出，

In [69]:
with open('./submit_7.json', 'w', encoding='utf8') as f:
    json.dump(questions, f, indent=4, ensure_ascii=False)